In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../scraper/src/data/new_sysbo.csv')

In [3]:
sysbo_df = df.copy()

In [4]:
sysbo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20542 entries, 0 to 20541
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              20542 non-null  int64  
 1   number                  20542 non-null  int64  
 2   Unnamed: 0.1            20542 non-null  int64  
 3   id                      20542 non-null  int64  
 4   item_number             20542 non-null  int64  
 5   name                    20542 non-null  object 
 6   name2                   16837 non-null  object 
 7   price_with_VAT          20542 non-null  float64
 8   pant                    1236 non-null   float64
 9   volume_ml               20542 non-null  float64
 10  price_per_liter         20542 non-null  float64
 11  start_sale              20542 non-null  object 
 12  expired                 20542 non-null  int64  
 13  item_group              20542 non-null  object 
 14  type                    12488 non-null

In [5]:
# Translation of groups into english.
group_translation = {
    'Rött vin': 'Red wine',
    'Öl': 'Beer',
    'Vitt vin': 'White vine',
    'Mousserande vin': 'Sparkling wine',
    'Whisky': 'Whisky',
    'Rom': 'Rum',
    'Rosévin': 'Rosé',
    'Aperitif och dessert': 'Appetizer and dessert',
    'Likör': 'Liquor',
    'Gin och Genever': 'Gin and Jenever',
    'Blandlådor vin': 'Mixed box',
    'Akvavit och Kryddat brännvin': 'Aquavit and Spicy brandy',
    'Cider': 'Cider',
    'Cognac': 'Cognac',
    'Vodka och Brännvin': 'Vodka and Brandy',
    'Alkoholfritt': 'Non-alcoholic',
    'Grappa och Marc': 'Grappa and Marc',
    'Sake': 'Sake',
    'Vermouth': 'Vermouth',
}

In [6]:
items = {
    'number': sysbo_df['number'],
    'name': sysbo_df['name'] + ' ' + sysbo_df['name2'].fillna(''),
    # ABV transformation to numerical feature.
    'abv': pd.to_numeric(sysbo_df['abv'].str.replace('%', '')),
    'price': sysbo_df['price_with_VAT'],
    'volume_ml': sysbo_df['volume_ml'],
    'price_per_liter': sysbo_df['price_per_liter'],
    'group': sysbo_df['item_group'].map(lambda x: group_translation.get(x, 'Other')),
    'type': sysbo_df['type'],
    'style': sysbo_df['style'],
    'country': sysbo_df['origin_country'],
    'bitterness': sysbo_df['scraped_beska'],
    'fullness': sysbo_df['scraped_fyllighet'],
    'sweetness': sysbo_df['scraped_sötma'],
}

In [7]:
items_df = pd.DataFrame(items)
items_df.set_index('number', inplace=True)
items_df.head()

,name,abv,price,volume_ml,price_per_liter,group,type,style,country,bitterness,fullness,sweetness
number,,,,,,,,,,,,
101,Renat,37.5,207.0,700.0,295.71,Vodka and Brandy,Vodka,NaN,Sverige,NaN,NaN,NaN
7548901,Arpepe Valtellina Superiore Stella Retica,13.5,381.0,750.0,508.00,Red wine,NaN,NaN,Italien,NaN,NaN,NaN
7521801,Purcari Freedom Blend,13.5,198.0,750.0,264.00,Red wine,NaN,NaN,Moldavien,NaN,NaN,NaN
8936603,Midas Golden Pilsner,4.9,27.9,330.0,84.55,Beer,Ljus lager,Pilsner - tjeckisk stil,Sverige,6.0,6.0,1.0
8780501,Josés Tequila Blanco,40.0,359.0,700.0,512.86,Other,Tequila,NaN,Mexiko,NaN,NaN,NaN


In [8]:
# Remove duplicate names.
to_keep_df = items_df.reset_index().groupby('name').first()
mask = to_keep_df['number']
items_df = items_df.loc[items_df.index.intersection(mask)]

In [9]:
items_df.to_csv('../scraper/src/data/sysbo.csv')